In [1]:
#ランタイムのタイプをGPUに変更
!pip install langchain
!pip install -U langchain-community
!pip install sentence-transformers
!pip install -U --quiet langchain-google-genai pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 k

In [2]:
import os
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

In [38]:
import pickle
path_w = '/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_texts.pkl'
with open(path_w, 'rb') as f:
    texts = pickle.load(f)
path_w1 = '/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_metadatas.pkl'
with open(path_w1, 'rb') as f:
    metadatas = pickle.load(f)
path_w2 = '/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_yojigen.pkl'
with open(path_w2, 'rb') as f:
    yojigen = pickle.load(f)

In [4]:
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

<ipython-input-4-7d781a8d0969>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
 

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 33.0 MB/s eta 0:00:00


In [42]:
store = FAISS.load_local('/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_vectorstore', embedding,allow_dangerous_deserialization=True)

In [53]:
def yjsearch(en):
  a = []
  for i,y in enumerate(yojigen):
    if not y <= en:
      a.append(str(i))
  return a

In [55]:
vectorstore = FAISS.load_local('/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_vectorstore', embedding,allow_dangerous_deserialization=True)
en = {'月1','月2','月3'}
vectorstore.delete(yjsearch(en))
retriever = vectorstore.as_retriever()
#retrieverの指定　デフォルトは類似度検索
#引数をsearch_kwargs={'filter': {'授業形態':'講義'}}とすると絞り込める
template = """
ユーザー:Questionに当てはまる科目の概要を述べてください。

{context}

{question}

システム:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
    template_format="f-string"
)
#プロンプトの作成

os.environ['GOOGLE_API_KEY'] = '*****************************'
#https://ai.google.dev/からAPIのキーを取得
llm = ChatGoogleGenerativeAI(model='gemini-pro')
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
)

q = "成績評価がレポートで、地理について学べる科目はなんですか"
ans = qa.invoke(q)

print(ans)

docs = ans['source_documents']
print('-----------------------------')
for d in docs:
    print(d.metadata['科目名'])

for d in docs:
    print()
    print(d.page_content)
    print(d.metadata['URL'])
    print(d.metadata['曜時限'])



> Entering new RetrievalQA chain...

> Finished chain.
{'query': '成績評価がレポートで、地理について学べる科目はなんですか', 'result': '地理学(特殊講義)', 'source_documents': [Document(metadata={'学部': '全学共通', '群': '人文・社会科学', '分野': '地域・文化', 'URL': 'https://www.k.kyoto-u.ac.jp/external/open_syllabus/la_syllabus?lectureNo=53720', '科目ナンバリング': 'U-LAS05 10009 LJ23', '所属部局、職名、氏名': '人間・環境学研究科教授小島泰雄', '科目名': '地域地理学 ', '英 訳': 'RegionalGeography', '使用言語': '日本語', '単位数': '2単位', '授業形態': '講義', '開講年度・開講期': '2024・前期', '配当学年': '全回生', '対象学生': '全学向', '曜時限': '月2'}, page_content='科目名は 地域地理学 。授業の概要・目的は 地域への地理学的アプローチ。人びとの暮らしを支える地域を、地理学はどのように捉えてきたのか、そして地誌をいかに書いてきたのかについて考えてゆく。到達目標は 地域地理学の実際の研究過程、地理教育、地域論、地誌の伝統、さらに学史を考えることで、地域地理学の基本的な事項を理解する。授業計画と内容は 以下のような課題について、1課題あたり1〜2週の授業をする予定である。授業回数は15回、最終回はフィードバック。（1）地域地理学と地理学：地域とは何か？（2）地域を考える：地域を共感的に理解するとは？（3）地域を調べる：なぜフィールドにでかけるのか？（4）地域を伝える：ステレオタイプ化を避けるためには？（5）等質地域：地域はどのように把握されてきたのか？（6）機能地域：もう一つの地域の捉え方とは？（7）風土記：なぜ地誌は書かれたのか？（8）方志：中国で書き継がれてきた地域とは？（9）近代と中国地誌：近代に中国をいかに書いてきたのか？（10）近代地理学と地域：科学としての地理学の形成の過程とは